In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ultralytics 

In [3]:
from ultralytics import YOLO
import os

print("🚀 Starting 2-STAGE YOLO Training on Kaggle...")

DATA_YAML = '/kaggle/input/convolveyolo/dataset/data_kaggle.yaml'

# ==========================================
# STAGE 1: LINEAR PROBE (Frozen Backbone)
# ==========================================
print("\n❄️ STAGE 1: Training Head Only (Frozen Backbone)...")
model = YOLO('yolo11s.pt') 

results_s1 = model.train(
    data=DATA_YAML,
    epochs=50,              # Increased to 30 to ensure head converges
    imgsz=1024,
    batch=16,
    name='stage1_frozen',
    device='0',
    
    # EARLY STOPPING (Stage 1)
    patience=8,            # Stop if no improvement for 10 epochs
    
    # FREEZE BACKBONE
    freeze=10,              # Freeze first 10 layers (Backbone)
    
    optimizer='AdamW',
    lr0=0.001,
    project='runs/detect'
)

# Get best weights from Stage 1
stage1_weights = f"runs/detect/stage1_frozen/weights/best.pt"
print(f"✅ Stage 1 Complete. Weights: {stage1_weights}")


# ==========================================
# STAGE 2: FINE-TUNING (Unfrozen)
# ==========================================
print("\n🔥 STAGE 2: Fine-Tuning Entire Model (Unfrozen)...")

# Load Stage 1 weights
model = YOLO(stage1_weights)

results_s2 = model.train(
    data=DATA_YAML,
    epochs=120,             # Long run
    imgsz=1024,
    batch=8,                # Lower batch for full model if needed
    name='stage2_finetune',
    device='0',
    
    # EARLY STOPPING (Stage 2)
    patience=30,            # Stop if no improvement for 20 epochs

    # UNFREEZE ALL
    freeze=0,
    
    # Low LR for fine-tuning
    optimizer='AdamW',
    lr0=0.0001,             # 10x smaller learning rate
    lrf=0.01,
    dropout=0.15,           # Slight dropout for regularization
    
    # Standard Augmentations
    degrees=5.0,
    mosaic=0.5,
    project='runs1/detect'
)

print(f"✅ Stage 2 Complete! Best weights: runs/detect/stage2_finetune/weights/best.pt")


🚀 Starting 2-STAGE YOLO Training on Kaggle...

❄️ STAGE 1: Training Head Only (Frozen Backbone)...
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/convolveyolo/dataset/data_kaggle.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=stage1_froze

FileNotFoundError: [Errno 2] No such file or directory: 'runs/detect/stage1_frozen/weights/best.pt'

In [7]:
stage1_weights = f"/kaggle/working/runs/detect/runs/detect/stage1_frozen/weights/best.pt"

In [8]:

# ==========================================
# STAGE 2: FINE-TUNING (Unfrozen)
# ==========================================
print("\n🔥 STAGE 2: Fine-Tuning Entire Model (Unfrozen)...")

# Load Stage 1 weights
model = YOLO(stage1_weights)

results_s2 = model.train(
    data=DATA_YAML,
    epochs=120,             # Long run
    imgsz=1024,
    batch=8,                # Lower batch for full model if needed
    name='stage2_finetune',
    device='0',
    
    # EARLY STOPPING (Stage 2)
    patience=30,            # Stop if no improvement for 20 epochs

    # UNFREEZE ALL
    freeze=0,
    
    # Low LR for fine-tuning
    optimizer='AdamW',
    lr0=0.0001,             # 10x smaller learning rate
    lrf=0.01,
    dropout=0.15,           # Slight dropout for regularization
    
    # Standard Augmentations
    degrees=5.0,
    mosaic=0.5,
    project='runs1/detect'
)


🔥 STAGE 2: Fine-Tuning Entire Model (Unfrozen)...
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/convolveyolo/dataset/data_kaggle.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.15, dynamic=False, embed=None, epochs=120, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/working/runs/detect/runs/detect/stage1_frozen/weights/best.pt, momentum=0.937, mosaic=0.5, multi_scale=0.0, name=